In [10]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
from typing import Any, Optional, List
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()

from ccxt.base.exchange import Exchange
from threading import Thread, Event, Lock
from queue import Queue
# from multiprocessing import Queue #as Queue

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV
from qubx.utils.pandas import scols, ohlc_resample
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64
from qubx.impl.ccxt_connector import CCXTConnector, CtrlChannel, BinanceQV
from qubx.utils.pandas import ohlc_resample

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# Listen to data

In [3]:
class RunThread(Thread):
    def __init__(self, channel: CtrlChannel):
        self.result = None
        self.ch = channel
        self.loops = []
        super().__init__()

    def add(self, func, *args, **kwargs):
        self.loops.append(func(self.ch, *args, **kwargs))

    async def run_loop(self):
        self.result = await asyncio.gather(*self.loops)

    def run(self):
        self.ch.control.set()
        asyncio.run(self.run_loop())

    def stop(self):
        self.ch.control.clear()
        self.ch.queue.put((None, None)) # send sentinel


async def listen_to_trades(channel: CtrlChannel, exchange: Exchange, symbols: List[str]):
    method = 'watchTrades'
    if exchange.has[method]:
        # lock = channel.lock
        while channel.control.is_set():
            try:
                # c_time = exchange.iso8601(exchange.milliseconds())
                trades = await exchange.watch_trades_for_symbols(symbols)
                # lock.acquire()
                for trade in trades:
                    # trade = trades[-1]
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s = trade['symbol']
                    info = trade['info']
                    price = trade['price']
                    m = info['m']
                    amnt = trade['amount'] # in base currency
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
                # lock.release()
                # print(trade, flush=True)
                # print(f"\t[{c_time}] {s} : {str(tas)}", flush=True)
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await exchange.close()
                raise e
    else:
        raise Exception(exchange.id + ' ' + method + ' is not supported or not implemented yet')


async def listen_to_orderbook(channel: CtrlChannel, exchange: Exchange, symbols: List[str], orderbook_max_levels_limit=5):
    lock = channel.lock
    while channel.control.is_set():
        try:
            orderbook = await exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
            # - check what this time is ?
            t_ns = orderbook['timestamp'] * 1_000_000
            s = orderbook['symbol']
            b, a = orderbook['bids'], orderbook['asks']
            q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
            # lock.acquire()
            channel.queue.put((s, q))
            # lock.release()
            # c_time = exchange.iso8601(exchange.milliseconds())
            # print(f"[{c_time}] {s} : {str(q)}", flush=True)
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e
 

async def receiver(channel: CtrlChannel):
    sers = {}
    print("START LISTENING", flush=True)
    T, Q = 0, 0
    while channel.control.is_set():
        s, data = channel.queue.get()
        if isinstance(data, Quote):
            if not (Q % 100):
                print(s, data, flush=True)
                Q = 0
            Q += 1
        if isinstance(data, Trade):
            if s not in sers:
                sers[s] = TimeSeries(s, '1Min')
            ts = sers[s]
            ts.update(data.time, data.price)
            if not (T % 100):
                print(s, data, flush=True)
                T = 0
            T += 1
        if isinstance(data, Bar):
            # print(f"{s} {pd.Timestamp(data.time, unit='ns')}: {data}", flush=True)
            if s not in sers:
                sers[s] = {}
            sers[s][data.time] = data

    print(f"STOP LISTENING: {str(sers)}", flush=True)
    return sers 

In [4]:
def prefetch_ohlcs(exchange: Exchange, symbol: str, timeframe: str, nbarsback: int):
    assert nbarsback > 1
    start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)
    return ohlcv

async def listen_to_ohlcv(channel: CtrlChannel, exchange: Exchange, 
                          symbol: List[str], timeframe: str, nbarsback: int):
    # - check if we need to load initial 'snapshot'
    if nbarsback > 1:
        ohlcv = await prefetch_ohlcs(exchange, symbol, timeframe, nbarsback)
        for oh in ohlcv:
            b = Bar(oh[0] * 1_000_000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
            channel.queue.put((symbol, b))

    while channel.control.is_set():
        try:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe)
            for oh in ohlcv:
                b = Bar(oh[0] * 1000000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
                channel.queue.put((symbol, b))
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

In [5]:
symbs = ['1INCHUSDT', 'AGLDUSDT', 'APEUSDT', 'ARBUSDT', 'AUCTIONUSDT', 'BANDUSDT', 'BATUSDT', 'C98USDT', 'COMPUSDT', 'CYBERUSDT',
        'DYDXUSDT', 'ENSUSDT', 'FXSUSDT', 'HFTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'LDOUSDT', 'LQTYUSDT', 'LTCUSDT',
        'MASKUSDT', 'MBOXUSDT', 'MINAUSDT', 'MTLUSDT', 'OGNUSDT', 'OPUSDT', 'PEOPLEUSDT', 'POLSUSDT', 'PYRUSDT', 'QNTUSDT', 'RADUSDT',
        'RDNTUSDT', 'RIFUSDT', 'RLCUSDT', 'SEIUSDT', 'SPELLUSDT', 'SSVUSDT', 'STGUSDT', 'STXUSDT', 'SUIUSDT', 'UMAUSDT', 'WAVESUSDT']

In [6]:
chan = CtrlChannel('ticks')
reader = RunThread(chan)
reader.add(receiver)
reader.start()

START LISTENING


In [7]:
# bu = cxp.binanceusdm()
# bu = cxp.binance()
bu = BinanceQV()
# bu = getattr(cxp, 'binanceqv')()
t1 = RunThread(chan)
for s in symbs:
    t1.add(listen_to_ohlcv, bu, s, '15m', 10)

# t1.add(listen_to_ohlcv, bu, 'BTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'ETHUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'LTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'SOLUSDT', '5m', 10)
# t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
t1.start()

In [10]:
reader.stop()
t1.stop()

In [ ]:
print(len(reader.result[0]), len(symbs))
reader.result[0]

# StrategyCtx

In [29]:
from qubx import logger

class ZeroStrat(IStrategy):
    param1: int = 0

    def on_start(self, ctx: StrategyContext):
        logger.info(f"> Started with param1 = {self.param1}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        logger.info(f"{event.time} -> {event}")
        xs = scols(*[ctx.ohlc(s, '1Min').pd()['close'].rename(s.symbol) for s in ctx.instruments])
        print(xs)

        return None

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> stopped")

In [22]:
cc = CCXTConnector('binance')
cc.time()

numpy.datetime64('2024-03-28T16:39:21.620535000')

In [ ]:
# cc.get_historical_ohlcs('BTCUSDT', '1d', 20)

In [23]:
ctx = StrategyContext(
    ZeroStrat(), dict(param1 = 1), cc, cc, 
    instruments=lookup.instruments[r'BINANCE:(BTC|ETH|LTC)USDT'],
    fees_spec='vip0_usdt', base_currency='USDT',
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=10),
    trigger=dict(type='bar', delay='-1s', timeframe='1Min')
)

2024-03-28 18:39:23.493 [ ℹ️ ] Set param1 -> 1


In [ ]:
ctx.start()

2024-03-28 18:39:26.363 [ ℹ️ ] Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 10} for 
	['BTCUSDT', 'ETHUSDT', 'LTCUSDT'] 
2024-03-28 18:39:26.364 [ ℹ️ ] > Started with param1 = 1
2024-03-28 18:39:26.365 [ ℹ️ ] > Data processor started
2024-03-28 18:39:26.368 [ ℹ️ ] Start processing market data
2024-03-28 18:39:26.370 [ ℹ️ ] > Market data subscribtions started


Task was destroyed but it is pending!
task: <Task cancelling name='Task-36' coro=<AiohttpClient.ping_loop() running at c:\Users\dmitr\devs\venvs\p311qbx\Lib\site-packages\ccxt\async_support\base\ws\aiohttp_client.py:124> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-44' coro=<AiohttpClient.ping_loop() running at c:\Users\dmitr\devs\venvs\p311qbx\Lib\site-packages\ccxt\async_support\base\ws\aiohttp_client.py:124> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-40' coro=<AiohttpClient.ping_loop() running at c:\Users\dmitr\devs\venvs\p311qbx\Lib\site-packages\ccxt\async_support\base\ws\aiohttp_client.py:124> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-86' coro=<AiohttpClient.ping_loop() running at c:\Users\dmitr\devs\venvs\p311qbx\Lib\site-packages\ccxt\async_support\base\ws\aiohttp_client.py:124> wait_for=<Future cancelled>>


In [26]:
ctx.stop()

2024-03-28 18:40:15.123 [ ℹ️ ] > stopped
2024-03-28 18:40:15.123 [ ℹ️ ] Market data processing finished


In [27]:
ctx.ohlc('BTCUSDT', '1Min')

                         open      high       low     close        volume  \
timestamp                                                                   
2024-03-28 16:30:00  70935.08  70976.27  70934.38  70976.26  1.409641e+06   
2024-03-28 16:31:00  70976.27  70976.27  70901.10  70901.11  2.127643e+06   
2024-03-28 16:32:00  70901.11  70925.11  70888.73  70910.12  9.900369e+05   
2024-03-28 16:33:00  70910.11  70998.99  70910.11  70937.99  1.627980e+06   
2024-03-28 16:34:00  70937.98  70940.68  70890.53  70925.78  1.513777e+06   
2024-03-28 16:35:00  70925.77  70971.27  70901.10  70952.11  9.592909e+05   
2024-03-28 16:36:00  70952.12  70961.99  70938.00  70955.29  8.306328e+05   
2024-03-28 16:37:00  70955.28  70994.13  70951.89  70982.14  1.622346e+06   
2024-03-28 16:38:00  70982.14  71043.30  70964.86  70970.32  5.434217e+06   
2024-03-28 16:39:00  70970.31  70985.36  70900.00  70919.98  1.189238e+06   
2024-03-28 16:40:00  70919.98  70932.00  70895.83  70895.83  3.284954e+05   

In [28]:
await cc.exch.close()

In [ ]:
# cc.close()

<hr/>

# Misc